# Caching and Dynamic Programming
Caching (Cache-ing, or making a cache, pronounced "cash-ing") is a very important technique in many areas of computing, and it just means "storing information for future use".

**Memo: Caching**  
Caching is storing information for future use.

We already know how to store information, using variables, so what makes caching different? Caching is about storing information that we might want to use multiple times, so we don't have to find it again. Many of the functions that we've looked at produce the same value every time you give them the same input. For example, executing the fibonacci function on 2 and 3 always produces 5. (Put another way, the 5th Fibonacci number is always 5.) If I ever need to use the 5th Fibonacci number (such as when computing the 6th Fibonacci number), instead of computing it, I could just check the variable where I stored the value.

This chapter is going to mostly explore different ways of computing Fibonacci numbers, using functions, loops, and caching. At the end will be a set of problems requiring you to apply these techniques to other functions and sequences, so that you can answer your own questions about them. But first, let's learn about time:

In [1]:
import time # This is an import statement - it lets us use code someone else wrote.
time_now = time.time() # time.time is a function that returns a number
print( time_now ) # specifically, it's the number of seconds since a specific date.

1570302520.2700317


**Problem: When is time measured from?**  
Do some math based on the number of seconds in a minute/hour/day/year to figure out what day time is measuring from. You can also search online for the answer, but that's less fun.

We can measure how long something takes for Python to run by measuring the difference between different times. Let's see how long a recursive definition of Fibonacci numbers takes to compute.

In [2]:
def fibonacci(n):
    """Computes the nth Fibonacci number"""
    # This is a "Doc String" - Think of it as another kind of comment.
    
    if n <= 0:
        return 0
    if n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

# import time was used in an earlier cell.
start = time.time()
print( fibonacci(35) ) # Actually calls the function and computes the fibonacci number.
end = time.time()

print(end - start)

9227465
11.458409070968628


This takes a little more than 6 seconds on my computer. That may not seem like a lot, but the time gets longer and longer very quickly. Why is this happening?

When we call fibonacci(35), that function calls fibonacci(34) and fibonacci(33). But fibonacci(34) calls fibonacci(33) and fibonacci(32), and fibonacci(33) calls fibonacci(32) and fibonacci(31). And each of those calls fibonacci several times, and so on and so forth. By the time all this is finished, fibonacci has been called tens of millions of times!

Maybe we can use the ideas of caching to speed this up? We can use a dictionary to store and lookup values that we might need later.

In [3]:
fibonacci2_cache = {0:0, 1:1} # This will be used to store cached values
def fibonacci2(n):
    """Computes the nth Fibonacci number"""
    if n in fibonacci2_cache: # Checks if the key is in the dictionary
        return fibonacci2_cache[n]
    else:
        fibonacci2_cache[n] = fibonacci2(n-1) + fibonacci2(n-2)
        return fibonacci2_cache[n]

start = time.time()
print( fibonacci2(35) ) # Actually calls the function and computes the fibonacci number.
end = time.time()

print(end - start)

9227465
0.0


Whoah! That's way faster!

Incidentally, this technique of caching the results of functions is called _memoization_ (memo-ization, or creating memos).

If we knew which values we needed to compute ahead of time, we could just generate them in the order they were needed. Here's an example of this process using loops:

In [4]:
fibonacci3_cache = {0:0, 1:1}
def fibonacci3(n):
    """Computes the nth Fibonacci number"""
    for i in range(2,n+1):
        fibonacci3_cache[i] = fibonacci3_cache[i-1] + fibonacci3_cache[i-2]
    return fibonacci3_cache[n]

start = time.time()
print( fibonacci3(35) ) # Actually calls the function and computes the fibonacci number.
end = time.time()

print(end - start)

9227465
0.0


# Decorators
Given any function, we could use similar techniques to cache information from that function and return it instead of doing the computation. But could we do this without modifying the original function? This is the idea behind a _decorator_, a function that takes in a function and returns a slightly modified version. Let's see how this could be used with our original fibonacci function:

In [5]:
def fibonacci(n):
    """Computes the nth Fibonacci number"""
    # This is a "Doc String" - Think of it as another kind of comment.
    
    if n <= 0:
        return 0
    if n == 1:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

def cache_function(function):
    the_cache = {} # A different copy is created every time cache_function is called/used.
    def lookup_first(input_number): # Define a new function that checks the cache first
        if input_number in the_cache:
            return the_cache[input_number]
        else:
            the_cache[input_number] = function(input_number)
            return the_cache[input_number]
    # The outer function (the decorator) returns the customized cache-checking function
    # Note that lookup_first remembers what function and the_cache referred to when it was created.
    # This is called a "closure".
    return lookup_first

start = time.time()
print( fibonacci(35) ) # Actually calls the function and computes the fibonacci number.
end = time.time()
print(end - start)

caching_fibonacci = cache_function(fibonacci) # Creates a new caching function

start = time.time()
print( caching_fibonacci(35) ) # This is still slow
end = time.time()
print(end - start)

start = time.time()
print( caching_fibonacci(35) ) # But if we call it again, the result is cached.
end = time.time()
print(end - start)

9227465
6.345026016235352
9227465
6.3259196281433105
9227465
0.0


This might be a little disappointing, since the inner fibonacci function still takes a long time the first time it runs. Is there any way to make the original fibonacci function call the faster cached version? Absolutely. We just have to use the function name it's looking for ("fibonacci" not "caching fibonacci").

In [6]:
start = time.time()
print( fibonacci(35) ) # Actually calls the function and computes the fibonacci number.
end = time.time()
print(end - start)

# This reassignment means that when fibonacci tries to call "fibonacci" recursively,
# it uses the caching function instead of the original.
fibonacci = cache_function(fibonacci)

start = time.time()
print( fibonacci(35) )
# Calling with a new number to show that it really is faster, and not just reusing the cache.
print( fibonacci(36) )
end = time.time()
print(end - start)

9227465
6.144596099853516
9227465
14930352
0.0


**Math: Pell Numbers**  
$$
P_n = 2\times P_{n-1} + P_{n-2}
$$  
So the first few Pell numbers are 0, 1, 2, 5, 12, 29...

**Problem: Caching Pell Numbers**  
Apply one of the caching techniques covered in this chapter in order to compute the first 100 Pell numbers

**Problem: Caching Statistics on the Collatz Flight time**  
Define the "flight time" of a number as the number of times the Collatz function must be applied to it before it reaches 1. By definition, flight_time(1) == 0. Use caching to efficiently compute the flight time of the first 100 natural numbers.

**Problem: Caching Prime Numbers**  
Use caching techniques to store prime numbers so that numbers can be factorized more easily.